In [68]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split as tts
from sklearn import preprocessing

In [69]:
data = pd.read_csv("weather_classification_data.csv")
print(data.isnull().sum())
data

Temperature             0
Humidity                0
Wind Speed              0
Precipitation (%)       0
Cloud Cover             0
Atmospheric Pressure    0
UV Index                0
Season                  0
Visibility (km)         0
Location                0
Weather Type            0
dtype: int64


,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30.0,64,7.0,16.0,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38.0,83,1.5,82.0,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27.0,74,17.0,66.0,overcast,990.67,1,Winter,2.5,mountain,Rainy
...,...,...,...,...,...,...,...,...,...,...,...
13195,10.0,74,14.5,71.0,overcast,1003.15,1,Summer,1.0,mountain,Rainy
13196,-1.0,76,3.5,23.0,cloudy,1067.23,1,Winter,6.0,coastal,Snowy
13197,30.0,77,5.5,28.0,overcast,1012.69,3,Autumn,9.0,coastal,Cloudy
13198,3.0,76,10.0,94.0,overcast,984.27,0,Winter,2.0,inland,Snowy


In [70]:
cloud = preprocessing.LabelEncoder()
cloud.fit(data["Cloud Cover"])
season = preprocessing.LabelEncoder()
season.fit(data["Season"])
location = preprocessing.LabelEncoder()
location.fit(data["Location"])
weather = preprocessing.LabelEncoder()
weather.fit(data["Weather Type"])

data["Cloud Cover"] = cloud.transform(data["Cloud Cover"])
data["Season"] = season.transform(data["Season"])
data["Location"] = location.transform(data["Location"])
data["Weather Type"] = weather.transform(data["Weather Type"])

In [71]:
data

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,3,1010.82,2,3,3.5,1,1
1,39.0,96,8.5,71.0,3,1011.43,7,1,10.0,1,0
2,30.0,64,7.0,16.0,0,1018.72,5,1,5.5,2,3
3,38.0,83,1.5,82.0,0,1026.25,7,1,1.0,0,3
4,27.0,74,17.0,66.0,2,990.67,1,3,2.5,2,1
...,...,...,...,...,...,...,...,...,...,...,...
13195,10.0,74,14.5,71.0,2,1003.15,1,2,1.0,2,1
13196,-1.0,76,3.5,23.0,1,1067.23,1,3,6.0,0,2
13197,30.0,77,5.5,28.0,2,1012.69,3,0,9.0,0,0
13198,3.0,76,10.0,94.0,2,984.27,0,3,2.0,1,2


In [72]:
data.max() - data.min()
x_train, x_test, y_train, y_test = tts(data.drop("Weather Type", axis=1), data["Weather Type"], train_size=0.8)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x_train.shape, y_train.shape

((10560, 10), (10560,))

In [73]:
scaler = preprocessing.StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [74]:
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(10, 25)
        self.l2 = nn.Linear(25, 50)
        self.l3 = nn.Linear(50, 20)
        self.l4 = nn.Linear(20, 5)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, x):
        x = self.relu(self.l1(x))
        x = self.relu(self.l2(x))
        x = self.relu(self.l3(x))
        x = self.softmax(self.l4(x))
        return x

In [75]:
ann = model()
ann = ann.to(device)
loss_fn = nn.CrossEntropyLoss()
optimiser = optim.SGD(ann.parameters(), lr=0.001)

In [76]:
for epoch in range(10):
    running_loss = 0
    for data,label in zip(x_train,y_train):
        data = torch.tensor(data, dtype=torch.float32)
        data = data.to(device)
        output = ann(data)

        estimated = torch.zeros(5, dtype=torch.float32)
        estimated[label] = 1
        estimated = estimated.to(device)
        loss = loss_fn(output,estimated)

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

    print(f"Epoch: {epoch}, Loss: {loss.item()}")

Epoch: 0, Loss: 1.556593894958496
Epoch: 1, Loss: 1.0974982976913452
Epoch: 2, Loss: 0.9664475917816162
Epoch: 3, Loss: 0.9558302164077759
Epoch: 4, Loss: 0.934395968914032
Epoch: 5, Loss: 0.9157981276512146
Epoch: 6, Loss: 0.9097847938537598
Epoch: 7, Loss: 0.9077333211898804
Epoch: 8, Loss: 0.906652569770813
Epoch: 9, Loss: 0.9059703946113586


## Testing

In [92]:
ann.eval()
correct = 0

for data, label in zip(x_test, y_test):
    data = torch.tensor(data, dtype=torch.float32)
    data = data.to(device)
    predict = ann(data)

    if predict.argmax().item() == label:
        correct += 1

print("Accuracy: %f" %((correct/x_test.shape[0]) * 100))
    

Accuracy: 88.068182
